In [ ]:
import pandas as pd 

In [ ]:
df_train = pd.read_csv('./new_data/train_accident_normalized_text.csv')
df_test = pd.read_csv('./new_data/test_accident_normalized_text.csv')

In [7]:
import torch

In [2]:
import torch.nn as nn

In [3]:
class RNN_LSTM_Base(nn.Module):
    def training_step(self, batch):
        samples, targets = batch
        outputs = self(samples.double())
        loss = nn.functional.mse_loss(outputs, targets)
        return loss

In [4]:
class VanillaRNN(RNN_LSTM_Base):
    def __init__(self, in_size, hid_size_rnn, hid_size_lin, out_size, n_layers=1):
        super(VanillaRNN, self).__init__()
# Define dimensions for the layers
        self.input_size = in_size
        self.hidden_size_rnn = hid_size_rnn
        self.hidden_size_lin = hid_size_lin
        self.output_size = out_size
        self.n_layers = n_layers
# Defining the RNN layer
        self.rnn = nn.RNN(in_size, hid_size_rnn, n_layers, batch_first=True)
# Defining the linear layer
        self.linear = nn.Linear(hid_size_lin, out_size)

In [5]:
def forward(self, x):
# x must be of shape (batch_size, seq_len, input_size)
    xb = x.view(x.size(0), x.size(1), self.input_size).double()
# Initialize the hidden layer's array of shape (n_layers*n_dirs, batch_size, hidden_size_rnn)
    h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_size_rnn, requires_grad=True).double()
# out is of shape (batch_size, seq_len, num_dirs*hidden_size_rnn)
    out, hn = self.rnn(xb, h0)
# out needs to be reshaped into dimensions (batch_size, hidden_size_lin)
    out = out.reshape(x.size(0), self.hidden_size_lin)
    out = nn.functional.relu(out)
# Finally we get out in the shape (batch_size, output_size)
    out = self.linear(out)
    return out

In [8]:
def fit(epochs, lr, model, train_loader, test_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
# Training phase
        model.train()
        for batch in train_loader:
            loss = model.training_step(batch)
# Calculate gradients from chain rule
            loss.backward()
# Apply gradient descent step
            optimizer.step()
# Remove gradients for next iteration
            optimizer.zero_grad()
    return 'Trained for {} epochs'.format(epochs)
